<hr color="#f0c6c6"/>
<h3><font color="#f0c6c6">ARM for Ingredients</font></h3>

<hr color="#f0c6c6"/>

In [1]:
import pandas as pd
df = pd.read_csv('../dataset/full_dataset.csv')

In [2]:
"""
    Since we will be running the ARM on ingredients, we need to remove quantifiers somehow.
"""
df['NER'] = df['NER'].apply(eval)

In [3]:
# Apply apriori algorithm
from mlxtend.preprocessing import TransactionEncoder

"""
    To be able to run the apriori algorithm, we first need to encode the ingredients
"""

te = TransactionEncoder()
te_ary = te.fit(df['NER'].values.tolist()).transform(df['NER'].values.tolist(), sparse=True)


In [4]:
df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

/var/folders/jt/f2yqrnv562q34hfjkkskqwzw0000gn/T/ipykernel_36282/215859792.py:1: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


In [5]:
from mlxtend.frequent_patterns import apriori,fpgrowth

frequent_itemsets = fpgrowth(
    df_encoded, min_support=0.01, use_colnames=True
)

In [15]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.75)

In [17]:
rules.sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2467,(cinnamon),(nutmeg),0.051608,0.022815,0.011871,0.230014,10.081815,0.010693,1.269095,0.949830
2466,(nutmeg),(cinnamon),0.022815,0.051608,0.011871,0.520303,10.081815,0.010693,1.977066,0.921843
2281,"(baking soda, eggs)","(salt, flour, vanilla)",0.027806,0.040189,0.010756,0.386837,9.625384,0.009639,1.565344,0.921738
2268,"(salt, flour, vanilla)","(baking soda, eggs)",0.040189,0.027806,0.010756,0.267643,9.625384,0.009639,1.327486,0.933630
2201,"(salt, flour, vanilla)","(baking soda, sugar)",0.040189,0.032962,0.011883,0.295680,8.970184,0.010558,1.373008,0.925724
...,...,...,...,...,...,...,...,...,...,...
168,(salt),"(flour, eggs, sugar, milk)",0.380587,0.015751,0.010526,0.027657,1.755891,0.004531,1.012245,0.694995
929,(eggs),"(flour, water)",0.178394,0.033084,0.010362,0.058082,1.755596,0.004460,1.026540,0.523844
926,"(flour, water)",(eggs),0.033084,0.178394,0.010362,0.313188,1.755596,0.004460,1.196261,0.445119
878,"(milk, egg)",(salt),0.019718,0.380587,0.013165,0.667682,1.754347,0.005661,1.863916,0.438636


In [18]:
# save these rules as ingredient_rules.csv
rules.to_csv('../dataset/ingredient_rules.csv', index=False)

____

In [2]:
df['ingredients'] = df['ingredients'].apply(lambda x: eval(x))

In [3]:
# check if 'allpurpose flour' exists in any of the ingredients,
# ingredients is a list of strings so we need to check if 'allpurpose flour' is in any of the strings
df['ingredients'].apply(
    lambda x: any(
        'allpurpose flour' in ingredient.lower() for ingredient in x
    )
).sum()

69

In [4]:
df['ingredients'].apply(
    lambda x: any(
        'all purpose flour' in ingredient.lower() for ingredient in x
    )
).sum()

14010

In [5]:
import pandas as pd
import re

# Function to remove common quantifiers
def remove_quantifiers(ingredient):
    # Patterns for common quantifiers and units
    # This list is not exhaustive and can be extended based on your dataset
    quantifier_patterns = [
        r'\b\d+/\d+\s',              # Fractions like '1/2'
        r'\b\d+\.\d+\s',             # Decimals like '1.5'
        r'\b\d+\s',                  # Whole numbers like '1'
        r'\bpinch(es)?\b',           # 'pinch' or 'pinches'
        r'\bdash(es)?\b',            # 'dash' or 'dashes'
        r'\bteaspoon(s)?\b',         # 'teaspoon' or 'teaspoons'
        r'\btablespoon(s)?\b',       # 'tablespoon' or 'tablespoons'
        r'\bcup(s)?\b',              # 'cup' or 'cups'
        r'\bpound(s)?\b',            # 'pound' or 'pounds'
        r'\bounce(s)?\b',            # 'ounce' or 'ounces'
        r'\bgram(s)?\b',             # 'gram' or 'grams'
        r'\bkilogram(s)?\b',         # 'kilogram' or 'kilograms'
        r'\bliter(s)?\b',            # 'liter' or 'liters'
        r'\bmililiter(s)?\b',        # 'mililiter' or 'mililiters'
        r'\bgallon(s)?\b',           # 'gallon' or 'gallons'
        r'\bquart(s)?\b',            # 'quart' or 'quarts'
        r'\bpint(s)?\b',             # 'pint' or 'pints'
        r'\bpackage(s)?\b',          # 'package' or 'packages'
        r'\bjar(s)?\b',              # 'jar' or 'jars'
        r'\bbottle(s)?\b',           # 'bottle' or 'bottles'
        r'\bslice(s)?\b',            # 'slice' or 'slices'
        r'\bwedge(s)?\b',            # 'wedge' or 'wedges'
        r'\bchunk(s)?\b',            # 'chunk' or 'chunks'
        r'\bclove(s)?\b',            # 'clove' or 'cloves'
        r'\bsprig(s)?\b',            # 'sprig' or 'sprigs'
        r'\bfillet(s)?\b',           # 'fillet' or 'fillets'
        r'\bstick(s)?\b',            # 'stick' or 'sticks'
        r'\bbunch(es)?\b',           # 'bunch' or 'bunches'
        r'\bhead(s)?\b',             # 'head' or 'heads' of lettuce for example
        r'\bbag(s)?\b',              # 'bag' or 'bags'
        r'\bbox(es)?\b',             # 'box' or 'boxes'
        r'\bclove(s)?\b',            # 'clove' or 'cloves'
        r'\blarge\b',                # 'large'
        r'\bsmall\b',                # 'small'
        r'\bmedium\b',               # 'medium'
        r'\bdiced\b',                # 'diced'
        r'\bminced\b',               # 'minced'
        r'\bchopped\b',              # 'chopped'
        r'\bsliced\b',               # 'sliced'
        r'\bjulienned\b',            # 'julienned'
        r'\bcrushed\b',              # 'crushed'
        r'\bgrated\b',               # 'grated'
        r'\bpeeled\b',               # 'peeled'
        r'\bcored\b',                # 'cored'
        r'\bshredded\b',             # 'shredded'
        r'\bmashed\b',               # 'mashed'
        r'\bto taste\b',             # 'to taste'
        r'\bas needed\b',            # 'as needed'
        r'\bpkg(s)?\b',              # for 'pkg' or 'pkgs'
        r'\btbsp(s)?\b',             # for 'tbsp' or 'tbsps'
        r'\btablespoon(s)?\b',
        r'\bteaspoon(s)?\b',
        r'\btsp(s)?\b',
        r'\boz\b',                   
        r'\bc\b',
        r'\btemperature\b',
        r'\broom\b',                      
    ]

    # Combine all patterns into a single regular expression
    combined_pattern = '|'.join(quantifier_patterns)

    # Remove the quantifiers
    cleaned_ingredient = re.sub(combined_pattern, '', ingredient, flags=re.IGNORECASE).strip()

    # Further cleanup for any stray numbers or symbols
    cleaned_ingredient = re.sub(r'[\d+]', '', cleaned_ingredient)
    cleaned_ingredient = re.sub(r'[^\w\s]', '', cleaned_ingredient)

    return cleaned_ingredient

# Apply the function to each ingredient in each list of ingredients
df['cleaned_ingredients'] = df['ingredients'].apply(lambda ingredients_list: [remove_quantifiers(ingredient) for ingredient in ingredients_list])

print(df['cleaned_ingredients'])

0          [ firmly packed brown sugar,  evaporated milk,...
1          [chipped beef cut up, boned chicken breasts, c...
2          [  frozen corn,   cream cheese cubed,  butter ...
3          [whole chicken,  cans chicken gravy,  can crea...
4          [ peanut butter,  graham cracker crumbs,  melt...
                                 ...                        
2231137    [chocolate hazelnut spread recommended Nutella...
2231138    [dozen eggs, paprika, salt and pepper to your ...
2231139    [Daikon radish, Sesame oil, White sesame seeds...
2231140    [apple cider, sugar, kosher salt plus more for...
2231141    [ground veal, sweet Italian sausage casings re...
Name: cleaned_ingredients, Length: 2231142, dtype: object


In [6]:
# join clean ingredients to a single string
df['cleaned_ingredients_joined'] = df['cleaned_ingredients'].apply(lambda x: ' '.join(x))
# Now apply word tokenization
import nltk
import nltk.data
#import preprocess string from gensim
from gensim.parsing.preprocessing import preprocess_string
df['cleaned_ingredients_joined'] = df['cleaned_ingredients_joined'].apply(lambda x: preprocess_string(x))

In [7]:
df['cleaned_ingredients_joined']

0          [firmli, pack, brown, sugar, evapor, milk, van...
1          [chip, beef, cut, bone, chicken, breast, cream...
2          [frozen, corn, cream, chees, cube, butter, cub...
3          [chicken, can, chicken, gravi, cream, mushroom...
4          [peanut, butter, graham, cracker, crumb, melt,...
                                 ...                        
2231137    [chocol, hazelnut, spread, recommend, nutella,...
2231138    [dozen, egg, paprika, salt, pepper, like, must...
2231139    [daikon, radish, sesam, oil, white, sesam, see...
2231140    [appl, cider, sugar, kosher, salt, plu, season...
2231141    [ground, veal, sweet, italian, sausag, case, r...
Name: cleaned_ingredients_joined, Length: 2231142, dtype: object

In [8]:
from mlxtend.preprocessing import TransactionEncoder

"""
    To be able to run the fpgrowth algorithm, we first need to encode the ingredients
"""

te = TransactionEncoder()
te_ary = te.fit(df['cleaned_ingredients_joined'].values.tolist()).transform(df['cleaned_ingredients_joined'].values.tolist(), sparse=True)

In [9]:
df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)

/var/folders/jt/f2yqrnv562q34hfjkkskqwzw0000gn/T/ipykernel_69904/215859792.py:1: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  df_encoded = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)


In [10]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(
    df_encoded, min_support=0.01, use_colnames=True
)

In [11]:
from mlxtend.frequent_patterns import association_rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [12]:
# filter rules with only max 2 items
rules['antecedent_len'] = rules['antecedents'].apply(lambda x: len(x))
rules_filtered = rules[rules['antecedent_len'] <= 2]

In [22]:
rules.sort_values(by=['confidence'], ascending=True)[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len
201063,(salt),"(freshli, oil, juic)",0.487162,0.010961,0.010000,0.020528,1.872749,0.004660,1.009767,0.908719,1
186721,(salt),"(wine, pepper, tomato)",0.487162,0.011889,0.010002,0.020531,1.726915,0.004210,1.008823,0.820791,1
94661,(salt),"(sugar, extract, brown)",0.487162,0.013945,0.010003,0.020532,1.472334,0.003209,1.006725,0.625551,1
87683,(salt),"(chees, drain, pepper)",0.487162,0.015965,0.010003,0.020534,1.286196,0.002226,1.004665,0.433887,1
188543,(salt),"(oil, thyme, fresh)",0.487162,0.012356,0.010004,0.020535,1.661940,0.003984,1.008350,0.776646,1
117242,(salt),"(ground, oliv, red, pepper, garlic)",0.487162,0.011360,0.010005,0.020538,1.807951,0.004471,1.009370,0.871402,1
110407,(salt),"(extract, egg, soda, vanilla, bake, flour)",0.487162,0.011705,0.010005,0.020538,1.754577,0.004303,1.009018,0.838592,1
140317,(salt),"(ground, oil, pepper, black, juic)",0.487162,0.010941,0.010008,0.020543,1.877707,0.004678,1.009804,0.911469,1
130910,(salt),"(sugar, flour, soda, shorten)",0.487162,0.012036,0.010009,0.020546,1.707109,0.004146,1.008689,0.807691,1
619,(salt),"(onion, milk, flour)",0.487162,0.012922,0.010010,0.020547,1.590061,0.003715,1.007785,0.723608,1


In [15]:
# find the rules that have antecedent as (egg,bake,pack,sugar)
rules[rules['antecedents'] == {'egg', 'bake', 'pack', 'sugar'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric,antecedent_len
14022,"(egg, sugar, bake, pack)",(flour),0.011599,0.257466,0.011227,0.967890,3.759295,0.008241,23.124888,0.742606,4
14056,"(egg, sugar, bake, pack)",(brown),0.011599,0.109258,0.011144,0.960703,8.792983,0.009876,22.667064,0.896674,4
14128,"(egg, sugar, bake, pack)","(flour, brown)",0.011599,0.046629,0.010789,0.930100,19.946617,0.010248,13.639154,0.961013,4


In [10]:
# union of all antecedents
antecedents = set()
for ant in oil_rules['antecedents']:
    antecedents = antecedents.union(ant)

NameError: name 'oil_rules' is not defined

In [38]:
antecedents

{'cup', 'extra', 'oliv', 'ounc', 'red', 'virgin'}